In [2]:
import copy
from copy import deepcopy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D , LSTM, Attention
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import*
import random as random
import time
import matplotlib.pyplot as plt

import Seth
from Seth import fetch_seth, Devices, Floorplan, get_mac_ids

ModuleNotFoundError: No module named 'Seth'

In [ ]:
def train():
    # dfs is a list of dataframes
# meta is a dataframe with meta data

#getting train data
    train_fp, train_meta = fetch_seth(
    Devices.lg,
    Floorplan.OFFICE,
    ci = 0,
    base_path="temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    train_fp = train_fp.sample(frac=1).reset_index(drop=True)
    train_aps = get_mac_ids(train_fp.columns)
    train_x = train_fp[train_aps].values
    train_x = (train_x + 100)/100
    train_y = (train_fp["label"]).values
    return train_x, train_y, train_aps

In [ ]:
def test(itr, train_aps):
    #getting test data
    test_fp, test_meta = fetch_seth(
    Devices.lg ,
    Floorplan.OFFICE,
    ci = itr,
    base_path="temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    test_aps = get_mac_ids(test_fp.columns)
    missing_aps = list(set(train_aps)-set(test_aps))
    test_fp[missing_aps] = -100
    test_x = test_fp[train_aps].values
    test_x = (test_x + 100)/100
    test_y = (test_fp["label"]).values
    return test_x, test_y


In [ ]:
def mean_cal(group):
    for i in range (0, len(group)):
        if group[i] < 0:
            group[i] = -group [i]
    mean_diff = np.mean(group)
    return mean_diff

In [ ]:
import keras.backend as K

def custom_mse(y_true, y_pred):
 
    # calculating squared difference between target and predicted values 
    loss = K.square(y_pred - y_true)  # (batch_size, 2)
    
    # multiplying the values with weights along batch dimension
    loss = loss * [0.3, 0.7]          # (batch_size, 2)
                
    # summing both loss values along batch dimension 
    loss = K.sum(loss, axis=1)        # (batch_size,)
    
    return loss

In [ ]:
train_x, train_y, train_aps = train()
callbacks = [tf.keras.callbacks.EarlyStopping(patience=300)]

error = []
mean_error = []
input_shape = train_x.shape[1] # input = 203
model = Sequential()
input = tf.keras.Input(shape=(input_shape,)) # enc = 190
encoded = Dense(190, activation='relu')(input)
decoded = Dense(input_shape, activation='softmax')(encoded) # recons = 203
autoencoder = Model(input,decoded)

opt = Adam(learning_rate=1e-4)

autoencoder.compile(loss=custom_mse,
            optimizer=opt,
            metrics=['accuracy'])
            

autoencoder.fit(train_x, train_x,
        epochs=1000,
        callbacks = callbacks,
        validation_split = 0.2,
        verbose=1)


In [ ]:
test_x, test_y = test(0, train_aps)

predicted = autoencoder.predict(train_x)


In [ ]:
train_x, train_y, train_aps = train()
callbacks = [tf.keras.callbacks.EarlyStopping(patience=300)]

error = []
mean_error = []
input_shape = train_x.shape[1] # input = 203
model = Sequential()
input = tf.keras.Input(shape=(input_shape,)) # enc = 190
encoded = Dense(190, activation='relu')(input)
encoded = Dense(175, activation='relu')(encoded)
decoded = Dense(190, activation='relu')(encoded)
decoded = Dense(input_shape, activation='softmax')(decoded) # recons = 203
autoencoder_2 = Model(input,decoded)

opt = Adam(learning_rate=1e-4)

autoencoder_2.compile(loss=custom_mse,
            optimizer=opt,
            metrics=['accuracy'])
            

autoencoder_2.fit(predicted, predicted,
        epochs=1000,
        callbacks = callbacks,
        validation_split = 0.2,
        verbose=1)


In [ ]:
test_x, test_y = test(0, train_aps)

predicted = autoencoder_2.predict(test_x)
